In [1]:
import json
from elasticsearch import Elasticsearch
from credentials import auth
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search, Q

In [4]:
u,p=auth()
hosts ="https://ctesm1.discoverdollar.com:41532/"
es = Elasticsearch(hosts,  timeout=60000, use_ssl=True, verify_certs=False, 
ssl_show_warn=False, raise_on_error=False,http_auth=(u,p))

In [5]:
index="cantire-2020-06"
body={"size":1000,
      "query":{
    "match_phrase":{
    "fileType": "Message/Email File"
    }}
}
res=es.search(index=index,body=body)

D:\Users\sourav.kumar\AppData\Local\Temp\2/ipykernel_9704/2159714140.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res=es.search(index=index,body=body)


In [6]:
es.ping()

True

In [7]:
import re
import json
def get_emails(txt):
    r=re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+',txt)
    return r

In [8]:
l=[]
for i in range(1000):
    s=set()
    
    content=res['hits']['hits'][i]['_source']['content']
    content.encode("ascii", "replace")
    
    s.update(get_emails(content))
    
    l.append(s)

In [9]:
l[0]

{'Ameera.Ladak@cantire.com',
 'Jennifer.Baird@cantire.com',
 'Laura.Allen@cantire.com',
 'Monique.Raymond@cantire.com',
 'anesthesia@utoronto.ca',
 'dbairdc313@Rogers.com',
 'dbairdc313@rogers.com'}

In [10]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [11]:
def check_similar(txt1,txt2):
    doc1 = nlp(txt1)
    doc2 = nlp(txt2)
    s=doc1.similarity(doc2)
    
    if s>=0.5:
        return s
    else:
        return 0
   


In [12]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|_\-\–<>;:]')
emails=re.compile('[\w.+-]+@[\w-]+\.[\w.-]+')
anchor=re.compile("<[^ ]*>")
cid=re.compile("cid[^ ]*")
sqbrac=re.compile('\[[^ ]*\]')
has=re.compile("#[0-9a-bA-B]*")

def clean_text(text):
    text=anchor.sub(" ",text)
    text=sqbrac.sub(" ",text)
    text=has.sub(" ",text)
    text=cid.sub(" ",text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text=" ".join(text.split())
    return text

In [13]:
def get_subject(text):
    lines=re.split('\n',text)
    subjects=[]
    pat=re.compile('Subject:')
    for i in lines:
        if re.match(pat,i):
            subjects.append(i[9:])
    return subjects
    

In [14]:
def sub_similar(sub1,sub2):
    m=0
    for i in sub1:
        for j in sub2:
            m=check_similar(i,j)
    return m
            

In [15]:
def is_similar(i,j):
    
    content1=res['hits']['hits'][i]['_source']['content']
    content2=res['hits']['hits'][j]['_source']['content']
    sub1=get_subject(content1)
    sub2=get_subject(content2)
    text1=clean_text(content1)
    text2=clean_text(content2)
    
    simi=sub_similar(sub1,sub2)
    
    return check_similar(text1,text2),simi

In [16]:
for i in range(1000):
    for j in range(i+1,1000):
        if l[j] and l[i] and (l[j].issubset(l[i]) or l[j].issuperset(l[i])) and len(l[i].intersection(l[j]))>=2:
            s,ss=is_similar(i,j)
            print(i,j,s,ss)
            
            

D:\Users\sourav.kumar\Anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
D:\Users\sourav.kumar\AppData\Local\Temp\2/ipykernel_9704/208563989.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  s=doc1.similarity(doc2)
D:\Users\sourav.kumar\AppData\Local\Temp\2/ipykernel_9704/208563989.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  s=doc1.similarity(doc2)


17 54 0 1.0
18 589 0 1.0
25 27 0 0
25 688 0 0
25 689 0 0
27 688 0 0
27 689 0 0
38 505 0 0
38 608 0 1.0
38 609 0 1.0
42 120 0 1.0
51 424 0 1.0
58 511 0 0
65 453 0 1.0
76 193 0 1.0
77 362 0 1.0
77 376 0 1.0
80 339 0 0
84 86 0 0
100 263 0 1.0
120 607 0 1.0
133 261 0 1.0
134 486 0 1.0
134 937 0 0
141 149 0 0
141 157 0 1.0
141 271 0 1.0
149 157 0 0
149 271 0 0
156 456 0 1.0
157 271 0 1.0
162 945 0 0
182 479 0 0
207 219 0 0
207 690 0 0
207 740 0 0
211 711 0 0
215 612 0 0
226 692 0 0


KeyboardInterrupt: 

In [17]:
l[38]

{'Annie.Fan2@cantire.com',
 'Joe.Li@cantire.com',
 'Lillian.Li@cantire.com',
 'PhaniKumar.Sudha@cantire.com',
 'beryl@mingdagroup.com',
 'eby@mingdagroup.com',
 'linda.luo@mingdagroup.com',
 'wangyiwen@mingdagroup.com'}

In [18]:
l[608]

{'Annie.Fan2@cantire.com',
 'Joe.Li@cantire.com',
 'Lillian.Li@cantire.com',
 'PhaniKumar.Sudha@cantire.com',
 'beryl@mingdagroup.com',
 'eby@mingdagroup.com',
 'linda.luo@mingdagroup.com',
 'wangyiwen@mingdagroup.com'}

In [22]:
print(res['hits']['hits'][38]['_source']['metaData'])

{'to_be_indexed': True, 'from': ['Joe.Li@cantire.com'], 'from_name': ['Joe Li'], 'to': ['linda.luo@mingdagroup.com'], 'to_name': ['linda.luo@mingdagroup.com'], 'cc': ['beryl@mingdagroup.com', 'wangyiwen@mingdagroup.com', 'eby@mingdagroup.com', 'Annie.Fan2@cantire.com', 'PhaniKumar.Sudha@cantire.com', 'Lillian.Li@cantire.com'], 'cc_name': ['=?utf-8?B?6LW16IiS6Zuv?=', '=?utf-8?B?5rGq6Im66Zuv?=', '=?utf-8?B?6LCi55uK6JCN?=', 'Annie.Fan', 'Phani Kumar Sudha', 'Lillian Li'], 'subject': "RE: 2D-320 / T & 3A T's replenishment programs", 'thread-topic': "2D-320 / T & 3A T's replenishment programs", 'thread-index': 'AdY9OXcdf4LzDgnAQMWagNw+OuphGg==', 'date': '06/08/2020', 'message-id': 'YTOPR0101MB0889E89CC541A176102381B498850@YTOPR0101MB0889.CANPRD01.PROD.OUTLOOK.COM'}


In [21]:
print(res['hits']['hits'][608]['_source']['metaData'])

{'to_be_indexed': True, 'message-id': '5ede1a29.1c69fb81.84a40.1f37SMTPIN_ADDED_BROKEN@mx.google.com', 'date': '06/08/2020', 'from': ['linda.luo@mingdagroup.com'], 'from_name': ['linda.luo@mingdagroup.com'], 'to': ['Joe.Li@cantire.com'], 'to_name': ['joe.li'], 'cc': ['beryl@mingdagroup.com', 'wangyiwen@mingdagroup.com', 'eby@mingdagroup.com', 'Annie.Fan2@cantire.com', 'PhaniKumar.Sudha@cantire.com', 'Lillian.Li@cantire.com'], 'cc_name': ['=?UTF-8?B?6LW16IiS6Zuv?=', '=?UTF-8?B?5rGq6Im66Zuv?=', '=?UTF-8?B?6LCi55uK6JCN?=', 'Annie.Fan2', 'Phani Kumar Sudha', 'Lillian Li'], 'subject': "Re: RE: 2D-320 / T & 3A T's replenishment programs"}


In [ ]:
import json


In [24]:
l='''17 54 0 1.0
18 589 0 1.0
25 27 0 0
25 688 0 0
25 689 0 0
27 688 0 0
27 689 0 0
38 505 0 0
38 608 0 1.0
38 609 0 1.0
42 120 0 1.0
51 424 0 1.0
58 511 0 0
65 453 0 1.0
76 193 0 1.0
77 362 0 1.0
77 376 0 1.0
80 339 0 0
84 86 0 0
100 263 0 1.0
120 607 0 1.0
133 261 0 1.0
134 486 0 1.0
134 937 0 0
141 149 0 0
141 157 0 1.0
141 271 0 1.0
149 157 0 0
149 271 0 0
156 456 0 1.0'''

In [26]:
ind=[]
for i in l.split('\n'):
    k=i.split()
    ind.append((int(k[0]),int(k[1])))

In [33]:
for i,j in ind:
    print(json.dumps(res['hits']['hits'][i]['_source']['metaData']))
    print()
    print(json.dumps(res['hits']['hits'][j]['_source']['metaData']))
    print()
    print("********************************************&&********************************************************")
    print()

{"to_be_indexed": true, "from": ["Misha.Mistry@cantire.com"], "from_name": ["Misha Mistry"], "to": ["AvanTroost@fluidmaster.com"], "to_name": ["Anthony van Troost"], "cc": ["JStephenson@fluidmaster.com"], "cc_name": ["John Stephenson"], "subject": "RE: [External] RE: Transition plan", "thread-topic": "[External] RE: Transition plan", "thread-index": "AdY0/yLCEskzTAOKTDWk5SGnMsB+BwAPhjlAASq7wNAAI2fBAAAAJc8wAAAadoA=", "date": "06/04/2020", "message-id": "YTBPR01MB3789F813E4749AF1ABCD1E208B890@YTBPR01MB3789.CANPRD01.PROD.OUTLOOK.COM", "in-reply-to": "MW3PR19MB42498393F20DF6C62046ECFBC2890@MW3PR19MB4249.namprd19.prod.outlook.com"}

{"to_be_indexed": true, "from": ["Misha.Mistry@cantire.com"], "from_name": ["Misha Mistry"], "to": ["AvanTroost@fluidmaster.com"], "to_name": ["Anthony van Troost"], "cc": ["JStephenson@fluidmaster.com"], "cc_name": ["John Stephenson"], "subject": "RE: Transition plan", "thread-topic": "Transition plan", "thread-index": "AdY0/yLCEskzTAOKTDWk5SGnMsB+BwAPhjlAASq7w